In [0]:
from pyspark.sql import SparkSession, functions as f

df_hosa = spark.read.parquet("/mnt/bronze/hosa/providers")

df_hosb = spark.read.parquet("/mnt/bronze/hosb/providers")

df_merged = df_hosa.unionByName(df_hosb)

display(df_merged)

df_merged.createOrReplaceTempView("providers")



In [0]:
%sql
use `hrcm-data-catalog`.silver;
create table if not exists providers(
  ProviderID string,
  FirstName string,
  LastName string,
  Specialization string,
  DeptID string,
  NPI long,
  datasource string,
  is_quarantined boolean
)
using delta;

In [0]:
%sql

truncate table `hrcm-data-catalog`.silver.providers;

In [0]:
%sql

insert into silver.providers
select 
  ProviderID,
  FirstName,
  LastName,
  Specialization,
  DeptID,
  cast(NPI as int) NPI,
  datasource,
  case when ProviderID is null or DeptID is null then true else false end as is_quarantined
  from providers;

In [0]:
df_silver = spark.sql("select * from `hrcm-data-catalog`.silver.providers")

df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/providers")